In [2]:
import ipympl
%matplotlib ipympl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [143]:
# Use the following data for this assignment:
np.random.seed(12345)
n_samples = 3650
years = [1992,1993,1994,1995]
df = pd.DataFrame([np.random.normal(32000,200000,n_samples), 
                   np.random.normal(43000,100000,n_samples), 
                   np.random.normal(43500,140000,n_samples), 
                   np.random.normal(48000,70000,n_samples)], 
                  index=years)
df

,0,1,2,3,4,5,6,7,8,9,...,3640,3641,3642,3643,3644,3645,3646,3647,3648,3649
1992,-8941.531897,127788.667612,-71887.743011,-79146.060869,425156.114501,310681.166595,50581.575349,88349.230566,185804.513522,281286.947277,...,171938.760289,150650.759924,203663.976475,-377877.158072,-197214.093861,24185.008589,-56826.729535,-67319.766489,113377.299342,-4494.878538
1993,-51896.094813,198350.518755,-123518.252821,-129916.759685,216119.147314,49845.883728,149135.648505,62807.672113,23365.577348,-109686.264981,...,-44566.520071,101032.122475,117648.199945,160475.622607,-13759.888342,-37333.493572,103019.841174,179746.127403,13455.493990,34442.898855
1994,152336.932066,192947.128056,389950.263156,-93006.152024,100818.575896,5529.230706,-32989.370488,223942.967178,-66721.580898,47826.269111,...,165085.806360,74735.174090,107329.726875,199250.734156,-36792.202754,-71861.846997,26375.113219,-29328.078384,65858.761714,-91542.001049
1995,-69708.439062,-13289.977022,-30178.390991,55052.181256,152883.621657,12930.835194,63700.461932,64148.489835,-29316.268556,59645.677367,...,-13901.388118,50173.686673,53965.990717,4128.990173,72202.595138,39937.199964,139472.114293,59386.186379,73362.229590,28705.082908


In [192]:
print('mean', np.mean(df.iloc[0]))
print('standard deviation', np.std(df.iloc[0]))
print('standard error', stats.sem(df.iloc[0]))
print('margin of error', stats.norm.ppf(.975)*stats.sem(df.iloc[i]))

mean 33312.1074755
standard deviation 200603.415985
standard error 3320.86631121
margin of error 2263.8101455


In [198]:
from scipy import stats
z = stats.norm.ppf(.975)
pdf = pd.DataFrame({'year':df.index.values, 
                    'mean':[df.iloc[i].mean() for i in range(4)], # m
                    'sem':[stats.sem(df.iloc[i]) for i in range(4)], # standard error of mean
                    'yerr':[z*stats.sem(df.iloc[i]) for i in range(4)], # margin of error
                    'ci_low':[df.iloc[i].mean() - (z*stats.sem(df.iloc[i])) for i in range(4)], # margin of error
                    'ci_high':[df.iloc[i].mean() + (z*stats.sem(df.iloc[i])) for i in range(4)], # margin of error
                   })
pdf

,ci_high,ci_low,mean,sem,year,yerr
0,39820.885843,26803.329108,33312.107476,3320.866311,1992,6508.778367
1,45054.055196,38669.663886,41861.859541,1628.701180,1993,3192.195655
2,44047.123549,34939.486333,39493.304941,2323.419534,1994,4553.818608
3,50007.361115,45479.740824,47743.550969,1155.026400,1995,2263.810146


In [11]:
import sys
stdout = sys.stdout
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = stdout

In [249]:
from matplotlib import cm
def update_bar_shading(data, y=4300):
    colour = []
    for i in range(4):
        prob =  stats.norm.cdf((y - data.iloc[i]['mean'])/(data.iloc[i]['sem']))
        bar_colour = cm.RdBu(prob)
        colour.append(bar_colour)
    return colour

update_bar_shading(pdf)

[(0.40392156862745099, 0.0, 0.12156862745098039, 1.0),
 (0.40392156862745099, 0.0, 0.12156862745098039, 1.0),
 (0.40392156862745099, 0.0, 0.12156862745098039, 1.0),
 (0.40392156862745099, 0.0, 0.12156862745098039, 1.0)]

In [251]:
import numpy as np
from __future__ import print_function

from matplotlib.patches import Rectangle
from matplotlib.lines import Line2D
from matplotlib.text import Text

from pylab import rcParams
rcParams['figure.figsize'] = 6, 5

bar_width = 0.8

fig, ax = plt.subplots(1, 1)
ax.set_title('Interactive Confidence Interval')

hline = ax.plot([1991.5, 1995.5], [y_mean, y_mean], 'k-', lw=1, picker=5)
htext = ax.text(1991.5, y_mean+100, '{:.2f}'.format(y_mean))


bars = ax.bar(pdf['year'], pdf['mean'], 
              color=colour,
              edgecolor='black',
              width=bar_width, 
              picker=True,
              yerr=pdf['yerr'],
              capsize=5)
print('bars {}'.format(bars))

for i in range(4):
    bars[i].set_gid(pdf['year'][i])

ax.tick_params(top='off', 
               bottom='on', 
               left='on', 
               right='off', 
               labelleft='on', 
               labelbottom='on')

ax.set_xticks(pdf['year'])
ax.set_yticks(np.arange(0, 50000+1, 10000))

for label in ax.get_xticklabels():
    label.set_picker(True)

for label in ax.get_yticklabels():
    label.set_picker(True)    


class DragHLine():
    """
    Simple object which allows us to click drag a horizontal line
    """
    def __init__(self, line, text, bars, data):
        self.line = line
        self.text = text
        self.text_offset = 300
        self.bars = bars
        
        self.data = data
        
        self.active = False
        
    def onpick(self, event):
        # disable when select a bar
        if isinstance(event.artist, Rectangle):
            self.active = False
        
    def onpress(self, event):
        self.active = True
        
    def onrelease(self, event):
        self.active = False
        
    def onmotion(self, event):
        if self.active == False:
            return
                
        if not event.ydata:
            return 
        
        # update horizonal line
        self.line.set_ydata([event.ydata, event.ydata])
        
        # update text
        self.text.set_y(event.ydata+self.text_offset)
        self.text.set_text('{:.2f}'.format(event.ydata))        
        
        # update bar colours
        colours = update_bar_shading(data=self.data, y=event.ydata)
        for colour, bar in zip(colours, self.bars):
            bar.set_color(colour)
            bar.set_edgecolor('black')
        
drag = DragHLine(line=hline[0], text=htext, bars=bars.patches, data=pdf)
        
fig.canvas.mpl_connect('pick_event', drag.onpick)
fig.canvas.mpl_connect('button_press_event', drag.onpress)
fig.canvas.mpl_connect('button_release_event', drag.onrelease)
fig.canvas.mpl_connect('motion_notify_event', drag.onmotion)
plt.show()


FigureCanvasNbAgg()

bars <BarContainer object of 4 artists>
